#### Сбор данных с сайта N1.ru


In [25]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd



#словарь с тэгами
d = {}
list_class = (
    "living-list-card-floor__item",
    "living-list-card__material living-list-card-material living-list-card__inner-block", 
    "living-list-card-price__item _object", 
    "living-list-card-newbuilding__title", 
    "living-list-card__area living-list-card-area living-list-card__inner-block",
    "card-title living-list-card__inner-block"
)

for cl in list_class:
    d[cl] = []

page = 0

# собирает словаь
while page in range(451): #на одной странице 25 объявлений

    url = 'https://ekaterinburg.n1.ru/kupit/kvartiry/vtorichka/region-Sverdlovskaya-oblast/?page=' + str(page)

    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'lxml')
    a = html_soup.find_all(class_="living-list-card")


    for cl in list_class:
        for i in a:
            if i.find(class_=cl) is None:
                d[cl].append(0)
            else:
                d[cl].append(i.find(class_=cl).text.strip())

    page += 1

    if page % 30 == 0:

        time.sleep(3)
        print(response)


pd.DataFrame(d).to_csv('out.csv', index=False) #сохраняет в csv файл
 



<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


#### Очистка/преобразование данных

In [44]:
import numpy as np

df1 = pd.read_csv('out.csv')


# переименование колонок
#floor - этаж, material - материал стен дома, price - цена квартиры, newbuilding - новостройка(сдана)/вторичка, square - площадь квартиры, rooms - количество комнат
df1.rename(columns={
        "living-list-card-floor__item":'floor', 
        "living-list-card__material living-list-card-material living-list-card__inner-block":'material', 
        "living-list-card-price__item _object":'price', 
        "living-list-card-newbuilding__title":'newbuilding', 
        "living-list-card__area living-list-card-area living-list-card__inner-block":'square',
        "card-title living-list-card__inner-block":'rooms',
}, inplace=True)



df1['price'] = df1['price'].str.replace(' ', '').astype(int)
a = df1['square'].str.split(' ', expand=True, )
df1['square'] = a[0].astype(int)
a = df1['floor'].str.split('/', expand=True, )
df1['floor'] = a[0].astype(int)
a = a[1].str.strip().str.split(' ', expand=True)

#информация о количестве этажей в доме
df1['hight'] = a[0].astype(int) 

#расположена на первом/последнем
df1['first'] = np.where(df1['floor'] == 1, 1, 0)
df1['last'] = np.where(df1['floor'] == df1['hight'], 1, 0)
a = df1['rooms'].str.strip().str.split(',', expand=True)

#улица расположения квартиры
df1['street'] = a[1]
df1['rooms'] = a[0]
a = a[2].str.strip().str.split('П', expand=True)

#номер дома
df1['house'] = a[0].replace(['\xa0'], '')

#стоимость квадратного метра
df1['kv'] = (df1['price']/df1['square']).astype(int)
df1.material.replace('Бетонные блоки - монолит', 'Бетонные блоки', inplace=True)
df1.material.replace('Кирпич - монолит', 'Кирпич', inplace=True)
df1.material.replace('Шлакоблоки', 'Бетонные блоки', inplace=True)
df1.newbuilding.replace('0', 'Вторичка', inplace=True)




df1.reset_index(inplace=True)


#фильтрует неактуальные данные
df1 = df1.query("material != '0' and material != 'Дерево'")
df1['address'] = (df1['street'] + ' ' + df1['house'].str.replace('\xa0', ''))
df1['address'] = df1.address.str.strip(' ')

df1.to_csv('total.csv', index=False)
